In [1]:
import numpy
import lsst.meas.base
import lsst.pex.config
import lsst.afw.table
import lsst.meas.algorithms
import lsst.meas.deblender
import lsst.pex.exceptions

In [2]:
class BoxFluxConfig(lsst.meas.base.SingleFramePluginConfig):
    
    width = lsst.pex.config.Field(
        dtype=int, default=30,
        doc="approximate width of rectangular aperture"
    )

    height = lsst.pex.config.Field(
        dtype=int, default=30,
        doc="approximate height of rectangular aperture"
    )

    
class BoxFluxTransform(lsst.meas.base.FluxTransform):

    def __init__(self, config, name, mapper):
        lsst.meas.base.FluxTransform.__init__(self, name, mapper)
        mapper.addMapping(mapper.getInputSchema().find(name + "_flag_edge").key)

        
@lsst.meas.base.register("ext_BoxFlux")
class BoxFluxPlugin(lsst.meas.base.SingleFramePlugin):

    ConfigClass = BoxFluxConfig
    
    FAILURE_EDGE = 1

    @classmethod
    def getExecutionOrder(cls):
        return cls.FLUX_ORDER

    @classmethod
    def getTransformClass(cls):
        return BoxFluxTransform
    
    def __init__(self, config, name, schema, metadata):
        lsst.meas.base.SingleFramePlugin.__init__(self, config, name, schema, metadata)

        # Get a FunctorKey that can quickly look up the "blessed" centroid value.
        self.centroidKey = lsst.afw.table.Point2DKey(schema["slot_Centroid"])

        # Add some fields for our outputs, and save their Keys.
        doc = "flux in a {0.width} x {0.height} rectangle".format(self.config)
        self.fluxKey = schema.addField(
            schema.join(name, "flux"), type=float, units="dn", doc=doc
        )
        self.fluxSigmaKey = schema.addField(
            schema.join(name, "fluxSigma"), type=float, units="dn",
            doc="1-sigma uncertainty for BoxFlux"
        )
        self.flagKey = schema.addField(
            schema.join(name, "flag"), type="Flag",
            doc="general failure flag for BoxFlux"
        )
        self.edgeFlagKey = schema.addField(
            schema.join(name, "flag", "edge"), type="Flag",
            doc="flag set when rectangle used by BoxFlux doesn't fit in the image"
        )
    
    
    def measure(self, measRecord, exposure):

        centroid = measRecord.get(self.centroidKey)
    
        # Create a single-pixel box
        point = lsst.afw.geom.Point2I(centroid)
        box = lsst.afw.geom.Box2I(point, point)

        # Grow the box in the desired size
        box.grow(lsst.afw.geom.Extent2I(self.config.width // 2, self.config.height // 2))

        # Horrible syntax to create a subimage.  Can't use [] because it doesn't pay
        # attention to xy0 :-(
        try: 
            subMaskedImage = exposure.getMaskedImage().Factory(
                exposure.getMaskedImage(),
                box,
                lsst.afw.image.PARENT
            )
        except lsst.pex.exceptions.LengthError as err:
            raise lsst.meas.base.MeasurementError(str(err), self.FAILURE_EDGE)

        # compute the flux by extracting and summing NumPy arrays.
        flux = subMaskedImage.getImage().getArray().sum()
        fluxSigma = subMaskedImage.getVariance().getArray().sum()**0.5

        measRecord[self.fluxKey] = flux
        measRecord[self.fluxSigmaKey] = fluxSigma
        
        
    def fail(self, measRecord, error=None):
        measRecord.set(self.flagKey, True)
        if error is not None:
            assert error.getFlagBit() == self.FAILURE_EDGE
            measRecord.set(self.edgeFlagKey, True)
            

In [3]:
schema = lsst.afw.table.SourceTable.makeMinimalSchema()
detectConfig = lsst.meas.algorithms.SourceDetectionConfig()
detectConfig.returnOriginalFootprints = False
detectConfig.thresholdValue = 10
detectTask = lsst.meas.algorithms.SourceDetectionTask(config=detectConfig, schema=schema)
deblendTask = lsst.meas.deblender.SourceDeblendTask(schema=schema)

In [4]:
measureConfig = lsst.meas.base.SingleFrameMeasurementConfig()
measureConfig.plugins.names.add("ext_BoxFlux")
measureTask = lsst.meas.base.SingleFrameMeasurementTask(config=measureConfig, schema=schema)

In [5]:
exposure = lsst.afw.image.ExposureF("coadd.fits") 
table = lsst.afw.table.SourceTable.make(schema)
detectResult = detectTask.run(table, exposure)
catalog = detectResult.sources
deblendTask.run(exposure, catalog, psf=exposure.getPsf())
measureTask.run(catalog, exposure)

In [6]:
from lsst.pipe.tasks.transformMeasurement import TransformTask

In [7]:
transformTask = TransformTask(measureConfig, catalog.schema, "src")
transformedCatalog = transformTask.run(catalog, exposure.getWcs(), exposure.getCalib())

In [8]:
for r1, r2 in zip(catalog, transformedCatalog):
    print r1.extract("ext_BoxFlux*"), r2.extract("ext_BoxFlux*")

{'ext_BoxFlux_flux': nan, 'ext_BoxFlux_fluxSigma': nan, 'ext_BoxFlux_flag': True, 'ext_BoxFlux_flag_edge': True} {'ext_BoxFlux_magErr': nan, 'ext_BoxFlux_flag': True, 'ext_BoxFlux_flag_edge': True, 'ext_BoxFlux_mag': nan}
{'ext_BoxFlux_flux': 6.820344924926758, 'ext_BoxFlux_fluxSigma': 0.785162323126388, 'ext_BoxFlux_flag': False, 'ext_BoxFlux_flag_edge': False} {'ext_BoxFlux_magErr': 0.1249906229397152, 'ext_BoxFlux_flag': False, 'ext_BoxFlux_flag_edge': False, 'ext_BoxFlux_mag': 24.91548415309326}
{'ext_BoxFlux_flux': 180.74533081054688, 'ext_BoxFlux_fluxSigma': 0.7971855755120664, 'ext_BoxFlux_flag': False, 'ext_BoxFlux_flag_edge': False} {'ext_BoxFlux_magErr': 0.004788689352930662, 'ext_BoxFlux_flag': False, 'ext_BoxFlux_flag_edge': False, 'ext_BoxFlux_mag': 21.357332282516587}
{'ext_BoxFlux_flux': 135.76541137695312, 'ext_BoxFlux_fluxSigma': 0.7971855755120664, 'ext_BoxFlux_flag': False, 'ext_BoxFlux_flag_edge': False} {'ext_BoxFlux_magErr': 0.006375211716047766, 'ext_BoxFlux_flag